In [1]:
!pip install catboost

     |████████████████████████████████| 69.2 MB 4.8 kB/s 


In [2]:
import warnings
warnings.filterwarnings('ignore')
import glob
import numpy as np
import pandas as pd

from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import StratifiedKFold
import random

In [3]:
!unzip /content/235745_parking_data.zip

Archive:  /content/235745_parking_data.zip
  inflating: 235745_parking_data/age_gender_info.csv  
  inflating: 235745_parking_data/sample_submission.csv  
  inflating: 235745_parking_data/test.csv  
  inflating: 235745_parking_data/train.csv  


In [4]:
train =pd.read_csv('/content/235745_parking_data/train.csv')
test = pd.read_csv('/content/235745_parking_data/test.csv')
submit = pd.read_csv('/content/235745_parking_data/sample_submission.csv')
population = pd.read_csv('/content/235745_parking_data/age_gender_info.csv')

In [5]:
test.loc[(test.단지코드 == 'C2411') & (test.자격유형.isnull()), '자격유형'] = 'A'

In [6]:
test.loc[(test.단지코드 == 'C2253') & (test.자격유형.isnull()), '자격유형'] = 'C'

In [7]:
train.loc[(train.지역 == '충청남도') & (train['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].isnull()), '도보 10분거리 내 지하철역 수(환승노선 수 반영)'] = 0
train.loc[(train.지역 == '경상남도') & (train['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].isnull()), '도보 10분거리 내 지하철역 수(환승노선 수 반영)'] = 0
train.loc[(train.지역 == '대전광역시') & (train['도보 10분거리 내 지하철역 수(환승노선 수 반영)'].isnull()), '도보 10분거리 내 지하철역 수(환승노선 수 반영)'] = 0

In [8]:
train.loc[train['도보 10분거리 내 버스정류장 수'].isnull(), '도보 10분거리 내 버스정류장 수'] = 3.0

In [9]:
train.loc[train.임대보증금.isnull()] = 0 
train.loc[train.임대료.isnull()] = 0 

test.loc[test.임대보증금.isnull()] = 0 
test.loc[test.임대료.isnull()] = 0

In [10]:
train.loc[(train.임대보증금 == '-')] = 0
train.loc[(train.임대료 == '-')] = 0

test.loc[(test.임대보증금 == '-')] = 0
test.loc[(test.임대료 == '-')] = 0

In [11]:
train.임대보증금 = train.임대보증금.astype('float64')
train.임대료 = train.임대료.astype('float64')

In [12]:
train['임대건물구분'] = train['임대건물구분'].astype('category').cat.codes
train['지역'] = train['지역'].astype('category').cat.codes
train['공급유형'] = train['공급유형'].astype('category').cat.codes
train['자격유형'] = train['자격유형'].astype('category').cat.codes

In [13]:
test.임대보증금 = test.임대보증금.astype('float64')
test.임대료 = test.임대료.astype('float64')

In [14]:
test['임대건물구분'] = test['임대건물구분'].astype('category').cat.codes
test['지역'] = test['지역'].astype('category').cat.codes
test['공급유형'] = test['공급유형'].astype('category').cat.codes
test['자격유형'] = test['자격유형'].astype('category').cat.codes

In [15]:
train_x = train.drop(['등록차량수', '단지코드'], axis = 1)
train_y = train['등록차량수']

test_x = test.drop(['단지코드'], axis = 1)

In [16]:
train = train.drop(['단지코드'], axis = 1)

In [17]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds=[]
for train_idx, valid_idx in skf.split(train, train['등록차량수']):
    folds.append((train_idx, valid_idx))

In [18]:
random.seed(42)
lgb_models={}
for fold in range(5):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train, X_valid, y_train, y_valid = train.drop(['등록차량수'],axis=1).iloc[train_idx].values, train.drop(['등록차량수'],axis=1).iloc[valid_idx].values,\
                                         train['등록차량수'][train_idx].values, train['등록차량수'][valid_idx].values 
    lgb = LGBMRegressor(n_estimators=1000)
    lgb.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], 
            early_stopping_rounds=30,
           verbose=100)
    lgb_models[fold]=lgb
    print(f'================================================================================\n\n')

====================================1============================================
Training until validation scores don't improve for 30 rounds.
[100]	training's l2: 855.038	valid_1's l2: 1845.31
[200]	training's l2: 204.801	valid_1's l2: 1140.33
[300]	training's l2: 78.1096	valid_1's l2: 954.777
[400]	training's l2: 36.4264	valid_1's l2: 873.448
[500]	training's l2: 19.815	valid_1's l2: 836.455
[600]	training's l2: 11.6582	valid_1's l2: 816.871
[700]	training's l2: 6.95782	valid_1's l2: 804.465
[800]	training's l2: 4.38528	valid_1's l2: 795.608
[900]	training's l2: 2.86259	valid_1's l2: 790.731
[1000]	training's l2: 1.88757	valid_1's l2: 787.005
Did not meet early stopping. Best iteration is:
[1000]	training's l2: 1.88757	valid_1's l2: 787.005


====================================2============================================
Training until validation scores don't improve for 30 rounds.
[100]	training's l2: 930.032	valid_1's l2: 3045.96
[200]	training's l2: 242.09	valid_1's l2: 2083.7


In [19]:
submit.iloc[:,1:]=0
pred_y = test['단지내주차면수']
pred_y = 0
for fold in range(5):
    pred_y += lgb_models[fold].predict(test_x)/5

In [20]:
pred_y = pd.DataFrame(pred_y)

In [21]:
pred_y['단지코드'] = test['단지코드']
pred_y

,0,단지코드
0,763.693661,C1072
1,784.052987,C1072
2,771.646273,C1072
3,769.544270,C1072
4,782.560116,C1072
...,...,...
1017,-0.000100,0
1018,217.209452,C2189
1019,213.738151,C2189
1020,226.200411,C2189


In [22]:
result = pred_y.groupby('단지코드').sum()
value = pred_y.groupby('단지코드').count()

result = result/value

In [23]:
pred_list = result.index.tolist()
for i in submit['code']:
    if(i in pred_list):
        submit.loc[submit.code == i, 'num'] = result.loc[result.index == i, 0][0]

In [26]:
submit['num']-=30

In [28]:
submit['num']=submit['num'].apply(lambda x:x if x>0 else 0)

In [30]:
submit.to_csv('20210727_lgb-30+.csv', index = False)